## Cu Nanoparticle calculation

In [ ]:
import os
from ase.io import read, write

In [ ]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 10 /opt/chemistry/conquest/dev/bin/Conquest'
os.environ['CQ_PP_PATH'] = '/opt/chemistry/conquest/dev/pseudo-and-pao/'
os.environ['CQ_GEN_BASIS_CMD'] = 'mpirun -np 1 /opt/chemistry/conquest/dev/bin/MakeIonFiles'

In [ ]:
from ase.cluster.cubic import FaceCenteredCubic

surfaces = [(1, 0, 0), (1, 1, 0), (1, 1, 1)]
layers = [6, 9, 3] #original code=[6, 9, 5]
lc = 3.61000 #lattice constant
struct = FaceCenteredCubic('Cu', surfaces, layers, latticeconstant=lc)

In [ ]:
diameter= struct.get_diameter()
n_layer = struct.get_layers()

print('diameter = ', diameter)
print('number of layers = ', n_layer)

In [ ]:
cell_length = diameter + 20
struct.set_cell([[cell_length, 0, 0],
                [0, cell_length, 0],
                [0, 0, cell_length]
               ]
              #,scale_atoms=(diameter/2, diameter/2, diameter/2)
              )

In [ ]:
# from ase.visualize import view
# view(struct)

In [ ]:
from my_visualize import print_xyz
print_xyz(struct)

In [ ]:
# Create workdir
working_directory = 'NP_layer' +str(layers[0]) +str(layers[1]) \
                  + str(layers[2]) +'_num'+ str(len(struct.positions)) + '_MP_p1_10kT'
print(working_directory)

# Test if `working_directory` exists ? If not create it
if ( not os.path.isdir(working_directory) ):
    os.makedirs(working_directory)

In [ ]:
write(working_directory + '/' + working_directory + '.cif', struct, format='cif')
write(working_directory + '/' + working_directory + '.traj', struct, format='traj')
write(working_directory + '/' + working_directory + '.xsf', struct, format='xsf')

In [ ]:
xc         = 'PBE'
basis_size = 'medium'
cutoff = 100.0
kpoints = [1,1,1]
charge = 1.0

In [ ]:
basis = {
         'Cu': {'gen_basis'            : True,
                'basis_size'           : basis_size,
                "pseudopotential_type": 'hamann',
                'xc'                   : xc,
                },
         'O' : { 'gen_basis'            : True,
                 'basis_size'           : basis_size,
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : xc },             
         'H' : { 'gen_basis'            : True,
                 'basis_size'           : basis_size,
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : xc },
         'C' : { 'gen_basis'            : True,
                 'basis_size'           : basis_size,
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : xc },         
          }
val_elec = {'Cu': 11, 'O': 6, 'H': 1, 'C': 4}

In [ ]:
from ase.calculators.conquest import Conquest

conquest_flags = {'IO.WriteOutToASEFile': True}

# Flags for atomic positions optimisation
conquest_flags.update({'AtomMove.TypeOfRun'   : 'static',  # optimization algorithm
                       'AtomMove.MaxForceTol' :  5e-3,   # 0.0005max Force component in Ha/bohr                       
                       'AtomMove.ReuseDM'     :  True,    
                       'AtomMove.AppendCoords':  True,
                       'General.netCharge'    : charge,
                       'Diag.SmearingType'    : 1,        # 0 = Fermi-Dirac, 1 = Methfessel-Paxton
                       'Diag.kT'              : 0.01,   # default = 0.001
                       #'AtomMove.WriteXSF T'  :  True,
                       })    

calc = Conquest(directory      = working_directory,
                grid_cutoff    = cutoff,
                scf_tolerance=5.0e-8,  # Ha
                self_consistent= True,
                xc    = xc,
                basis = basis,
                kpts  = kpoints,
                nspin = 1,
                **conquest_flags)

In [ ]:
# Create input files
struct.calc.write_input(struct)